In [1]:
!pip install impedance

In [2]:
import numpy as np
import pandas as pd

In [3]:
eis_data = pd.read_excel('EIS_data.xlsx')

print(eis_data)

       100019.51  255.73708     23.30298   100019.51.1   243.7702  \
0   79121.078000  257.96289    20.019239  79121.078000  245.99348   
1   62607.406000  259.59836    16.947983  62607.406000  247.96529   
2   49531.234000  261.08597    14.611738  49531.234000  249.89438   
3   39189.441000  262.57779    12.693474  39189.441000  251.68262   
4   31015.619000  263.25327    10.590907  31015.619000  252.77010   
5   24541.012000  264.08499     8.993812  24541.012000  254.11682   
6   19414.059000  264.76248     7.652000  19414.059000  254.92578   
7   15361.323000  265.23096     6.640215  15361.323000  255.80453   
8   12158.199000  265.81989     5.522952  12158.199000  256.27512   
9    9753.725600  266.34955     5.126089   9753.725600  256.76617   
10   7744.431200  266.85236     4.596840   7744.431200  257.32816   
11   6125.576700  266.78479     4.506677   6125.576700  257.79562   
12   4763.476100  267.66315     4.059433   4763.476100  257.96228   
13   3768.794200  267.55536     3.

In [4]:
no_head = eis_data.loc[1:,:]
print(no_head.head())

   100019.51  255.73708   23.30298  100019.51.1   243.7702  20.828268  \
1  62607.406  259.59836  16.947983    62607.406  247.96529  16.335632   
2  49531.234  261.08597  14.611738    49531.234  249.89438  14.531267   
3  39189.441  262.57779  12.693474    39189.441  251.68262  12.452054   
4  31015.619  263.25327  10.590907    31015.619  252.77010  10.822877   
5  24541.012  264.08499   8.993812    24541.012  254.11682   9.159061   

   100019.51.2  261.27975  37.719074  100019.51.3    ...       60.633286  \
1    62607.406  265.40744  32.645943    62607.406    ...       72.753716   
2    49531.234  265.40744  30.650309    49531.234    ...       82.187912   
3    39189.441  265.40744  27.802244    39189.441    ...       95.560585   
4    31015.619  265.40744  25.448105    31015.619    ...      111.643270   
5    24541.012  265.40744  22.161785    24541.012    ...      131.982970   

   100019.51.9  251.56194  78.337105  100019.51.10  251.56194.1  78.337105.1  \
1    62607.406  263.0961

In [7]:
freq = no_head.values
print(freq)

[[6.2607406e+04 2.5959836e+02 1.6947983e+01 ... 6.2607406e+04
  3.3442270e+02 1.0075674e+02]
 [4.9531234e+04 2.6108597e+02 1.4611738e+01 ... 4.9531234e+04
  3.4178629e+02 1.1664044e+02]
 [3.9189441e+04 2.6257779e+02 1.2693474e+01 ... 3.9189441e+04
  3.5099185e+02 1.3650568e+02]
 ...
 [1.5969276e-01 3.7202454e+02 2.9312554e+03 ... 1.5969276e-01
  1.9650215e+06 3.5636673e+06]
 [1.2635216e-01 3.8698108e+02 3.6916462e+03 ... 1.2635216e-01
  2.4179025e+06 4.1046570e+06]
 [1.0005087e-01 4.0554095e+02 4.6470522e+03 ... 1.0005087e-01
  2.9420453e+06 4.6797590e+06]]


In [ ]:
### Drop all the frequency colmun
col_remove = []
for col in no_head.columns:
    if 'U' != col[0]:
        col_remove.append(col) 
print(col_remove)

In [ ]:
impedance_raw = no_head.copy().drop(col_remove, axis = 1)
print(impedance_raw)

In [ ]:
###convert DataFrame to Array
impedance_array = impedance_raw.values
print(impedance_array)
print(type(impedance_array[0,0]))

In [ ]:
### define the function for the preparation of fitting
def prep_fit(data_set,col,freq):
    Z_data = data_set[:,col] - 1j*data_set[:,col+1]
    freq = freq[np.imag(Z_data) < 0]
    Z_data = Z_data[np.imag(Z_data) < 0]
    print(freq)
    print(Z_data)
    return freq, Z_data

In [ ]:
### define the fuctioon for fitting
from impedance.circuits import CustomCircuit

def do_fit(freq = freq, Z_data = Z_data, circuit = 'R0-p(R1,C1)-W1/W2-C2', initial_guess = [.01, .01, 100, .05, 0.1, 100]):
    circuit = CustomCircuit(circuit, initial_guess = initial_guess)
    circuit.fit(freq, Z_data)
    print(circuit)
    Z_fit = circuit.predict(freq)
    return circuit.parameters_, Z_fit

fit_result = do_fit(freq = freq, Z_data = Z_data, circuit ='R0-p(R2,C1)-W1/W2-C2', initial_guess = [.01, .01, 100, .05, 0.1, 100])

#print(circuit.parameters_)

In [ ]:
### plot the fitting and data spots:
def nyquist_polt(freq,Z_data, Z_fit):
    import matplotlib.pyplot as plt
    from impedance.plotting import plot_nyquist
    fig, ax = plt.subplots()
    plot_nyquist(ax, freq, Z_data, fmt='o')
    plot_nyquist(ax, freq, Z_fit, fmt='-')
    plt.legend(['Data', 'Fit'])
    plt.show()